In [1]:
import cudaq
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Function
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import shap
from lime.lime_tabular import LimeTabularExplainer
     

/home/sasuke/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cudaq.set_target("qpp-cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# ---------------------
# Quantum Operations
# ---------------------
def ry(theta, qubit):
    cudaq.ry(theta, qubit)

def rx(theta, qubit):
    cudaq.rx(theta, qubit)

In [4]:
# ---------------------
# Quantum Function
# ---------------------
class QuantumFunction(Function):
    def __init__(self, qubit_count: int, hamiltonian: cudaq.SpinOperator):
        @cudaq.kernel
        def kernel(qubit_count: int, thetas: np.ndarray):
            qubits = cudaq.qvector(qubit_count)
            ry(thetas[0], qubits[0])
            rx(thetas[1], qubits[0])
        self.kernel = kernel
        self.qubit_count = qubit_count
        self.hamiltonian = hamiltonian

    def run(self, theta_vals: torch.Tensor) -> torch.Tensor:
        theta_vals_np = theta_vals.cpu().numpy()
        qubit_count = [self.qubit_count for _ in range(theta_vals_np.shape[0])]
        results = cudaq.observe(self.kernel, self.hamiltonian, qubit_count, theta_vals_np)
        exp_vals = [results[i].expectation() for i in range(len(results))]
        return torch.tensor(exp_vals, dtype=torch.float32, device=device)

    @staticmethod
    def forward(ctx, thetas: torch.Tensor, quantum_circuit, shift) -> torch.Tensor:
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit
        exp_vals = ctx.quantum_circuit.run(thetas).reshape(-1, 1)
        ctx.save_for_backward(thetas, exp_vals)
        return exp_vals

    @staticmethod
    def backward(ctx, grad_output):
        thetas, _ = ctx.saved_tensors
        gradients = torch.zeros_like(thetas)
        for i in range(thetas.shape[1]):
            thetas_plus = thetas.clone()
            thetas_plus[:, i] += ctx.shift
            exp_vals_plus = ctx.quantum_circuit.run(thetas_plus)

            thetas_minus = thetas.clone()
            thetas_minus[:, i] -= ctx.shift
            exp_vals_minus = ctx.quantum_circuit.run(thetas_minus)

            gradients[:, i] = (exp_vals_plus - exp_vals_minus) / (2 * ctx.shift)

        return gradients * grad_output, None, None

# ---------------------
# Quantum Layer
# ---------------------
class QuantumLayer(nn.Module):
    def __init__(self, qubit_count: int, hamiltonian, shift: torch.Tensor):
        super(QuantumLayer, self).__init__()
        self.quantum_circuit = QuantumFunction(qubit_count, hamiltonian)
        self.shift = shift

    def forward(self, input: torch.Tensor):
        return QuantumFunction.apply(input, self.quantum_circuit, self.shift)

In [5]:

# ----------------------------------
#   Residual QNN (Skip Connections)
# ----------------------------------
class ResidualQNN(nn.Module):
    def __init__(self, input_dim):
        super(ResidualQNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)

        # Skip projection (from dimension 128 -> 64)
        self.skip_linear = nn.Linear(128, 64)

        # A quantum layer with 4 qubits
        self.quantum = QuantumLayer(
            qubit_count=4,
            hamiltonian=cudaq.spin.z(0),
            shift=torch.tensor(torch.pi / 2)
        )

        # Final output (assuming a single-value regression)
        self.fc_out = nn.Linear(1, 2)

    def forward(self, x):
        # Classical feedforward
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))

        # Residual block: skip from x -> fc3(x)
        residual = self.skip_linear(x)
        x = torch.relu(self.fc3(x))
        x = x + residual
        x = self.quantum(x).view(-1, 1)

        # Final linear output
        x = self.fc_out(x)
        return x

In [6]:
import pandas as pd
import numpy as np

def load_data(path):
    data = pd.read_csv(path)
    return data

dataset_path = "../data/noalpha.csv"  # Adjust to your CSV path
data = load_data(dataset_path)

# Identify only numeric columns
numeric_cols = data.select_dtypes(include=[np.number]).columns
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

features = ['RAdeg', 'DEdeg', 'e_RAdeg', 'e_DEdeg', 'RApeak',
            'DEpeak', 'Sint', 'e_Sint', 'Speak', 'e_Speak', 'rmspeak', 'e_rmspeak', 'PA']

target = ['thetamaj', 'thetamin']

X = data[features].values
y = data[target].values  # shape: (5762, 2)

# Normalize
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)  # keep it 2D for multi-output regression

# Convert to torch tensors
X = torch.tensor(X, dtype=torch.float32).to(device)
y = torch.tensor(y, dtype=torch.float32).to(device)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.1,
    random_state=42
)


In [8]:
# ---------------------
#   Training
# ---------------------
model = ResidualQNN(input_dim=len(features)).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.MSELoss().to(device)

epochs = 5
losses = []

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    y_hat_train = model(X_train)
    loss = loss_function(y_hat_train, y_train)  # No squeeze

    loss.backward()
    optimizer.step()

    losses.append(loss.item())
    std_loss = np.std(losses)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.6f}, Std Dev: {std_loss:.6f}")


Epoch 1/5, Loss: 1.013362, Std Dev: 0.000000
Epoch 2/5, Loss: 1.008444, Std Dev: 0.002459
Epoch 3/5, Loss: 0.998027, Std Dev: 0.006393
Epoch 4/5, Loss: 0.986318, Std Dev: 0.010386
Epoch 5/5, Loss: 0.975075, Std Dev: 0.014083


In [10]:
# ---------------------
#   Evaluation
# ---------------------
model.eval()
with torch.no_grad():
    y_hat_test = model(X_test)

# Convert torch tensors -> NumPy arrays
y_test_np = y_test.detach().cpu().numpy()
y_hat_test_np = y_hat_test.detach().cpu().numpy()

# Now pass these arrays to the scikit-learn functions
r2_score_value = -r2_score(y_test_np, y_hat_test_np)
mae_score_value = mean_absolute_error(y_test_np, y_hat_test_np)
rmse_score_value = np.sqrt(mean_squared_error(y_test_np, y_hat_test_np))
mse_score_value = mean_squared_error(y_test_np, y_hat_test_np)

# Then you can safely do your alpha computation
y_max = y_test_np.max()
y_min = y_test_np.min()
alpha = y_max - y_min

mae_accuracy = (1 - mae_score_value / alpha) * 100
rmse_accuracy = (1 - rmse_score_value / alpha) * 100
mse_accuracy = (1 - mse_score_value / alpha) * 100


In [11]:
print("\n--- Evaluation Metrics ---")
print(f"R^2 Score: {-r2_score_value:.4f}")
print(f"MAE: {mae_score_value:.4f} | Accuracy: {mae_accuracy:.2f}%")
print(f"RMSE: {rmse_score_value:.4f} | Accuracy: {rmse_accuracy:.2f}%")
print(f"MSE:  {mse_score_value:.4f} | Accuracy: {mse_accuracy:.2f}%")


--- Evaluation Metrics ---
R^2 Score: 0.0308
MAE: 0.6642 | Accuracy: 94.95%
RMSE: 1.0159 | Accuracy: 92.28%
MSE:  1.0320 | Accuracy: 92.16%


In [15]:
import os
import tempfile
import torch
with tempfile.NamedTemporaryFile(delete=False, suffix=".pt") as tmp_file:
    torch.save(model.state_dict(), tmp_file.name)
    model_size_mb = os.path.getsize(tmp_file.name) / (1024 * 1024)  # bytes to MB
    os.remove(tmp_file.name)  # Clean up

print(f"Model Size: {model_size_mb:.3f} MB")

Model Size: 0.206 MB
